# Use options

Options provided for taskbook can be created via "options" classes.

There are two categories of options:

* **ExperimentOptions**:
    These options are used to set some parameters for the experiment. For example, the number of experiment shots (`counts`),
    the averaging mode (`averaging_mode`), the acquisition type (`acquisition_type`) and so on. 

* **TaskBookOptions**:
    These options are used to set some parameters for the taskbook. These include the settings for operating the taskbook, such as `run_until` which specifies the task the taskbook should stop at.
    In addition, options for the constituent tasks of the taskbook can also be set by specifying the task name and the options for that task.



In [ ]:
from __future__ import annotations

from laboneq_applications.core.options import (
    BaseExperimentOptions,
    TuneupExperimentOptions,
)
from laboneq_applications.workflow import TaskBookOptions, task, taskbook

# Create a new experiment options class
The options for an experiment can be created by inheriting from `BaseExperimentOptions` class.

It is recommended to create options classes always with default values. This way, options can be used without specifying them, and the default values will be used.

In [ ]:
class NewExperimentOptions(BaseExperimentOptions):
    operand: int = 1

In [ ]:
opt = NewExperimentOptions()

In [ ]:
opt

The library provides a few standard options classes, such as `TuneupExperimentOptions`

In [ ]:
opt = TuneupExperimentOptions()
opt

# Create a new taskbook options class

Taskbook options classes are used to set the options for the taskbook and its constituent tasks. These classes must inherit from `TaskBookOptions`, as shown in the cell below. 

The options for the constituent tasks of the taskbook can be set by specifying the task name and the options class used for that task.
As in the case of the experiment options, it is recommended to create the taskbook options class with default values.

For example, in the cell below, the line `add: NewExperimentOptions = NewExperimentOptions()` sets the type `NewExperimentOptions` and default value `NewExperimentOptions()` for the options of task `add`:

In [ ]:
class NewTaskBookOptions(TaskBookOptions):
    add: NewExperimentOptions = NewExperimentOptions()
    multiply: NewExperimentOptions = NewExperimentOptions()

In [ ]:
taskbook_opt = NewTaskBookOptions()

In [ ]:
taskbook_opt.add.operand = 2

# Enable the options feature in the taskbook

The advantage of using the options feature of a taskbook is the automatic passing of options to the tasks and other features like run_until, etc.

Let's illustrate the concepts via an example. 

`mytaskbook` contains task `mytask` which takes in an argument and options. 

To use `mytaskbook` with options, we must provide the right types for the `options` argument in `mytaskbook`.

The supported types are:
- `TaskBookOptionsA | None`
- `Union[TaskBookOptionsA, None]`
- `Optional[TaskBookOptionsA]`

where `TaskBookOptionsA` is a subclass of `TaskBookOptions`.

From Python 3.10 onward, it is recommended to use `TaskBookOptionsA | None` to conform with the standard practice.

On Python 3.9, `from _future_ import annotations` must be imported to use `TaskBookOptionsA | None`.


In [ ]:
@task
def add(x, options: NewExperimentOptions):
    return x + options.operand


@task
def multiply(x, options: NewExperimentOptions):
    return x * options.operand

In [ ]:
@taskbook
def mytaskbook(options: NewTaskBookOptions | None = None):
    add(x=1)
    multiply(x=2)

# Disallowed types for options

When the type provided for the options includes a subclass of `TaskBookOptions`, we assume that users are attempting to use the options feature. Hence, if the specified type does not follow the form of the above-mentioned types, an error will be raised to inform the user about this, as shown below. 

Note: users are allowed to define and use their own options, passed as any standard Python type (`str`, `dict` etc.). See 'Manual handling of options' below. 

In [ ]:
# an error will be raised


@taskbook
def mytaskbook(options: NewTaskBookOptions | str):
    add(x=1)
    multiply(x=2)

# Run the taskbook with options

Create an options object and pass it to the taskbook when running it.

In [ ]:
taskbook_opt = NewTaskBookOptions()

In [ ]:
taskbook_opt.add.operand = 3
taskbook_opt.multiply.operand = 3

In [ ]:
res = mytaskbook(options=taskbook_opt)
res.tasks[1].output

In [ ]:
taskbook_opt.run_until = "add"

Note how the options for each task are automatically set by the taskbook options.

In [ ]:
res = mytaskbook(options=taskbook_opt)
res.tasks  # only the first task is executed

# Manual handling of options

For quick prototyping, it is possible to disable the automatic forwarding of `options` by the taskbook to its tasks. 

This can be done by not including `TaskBookOptions` or its subclasses for the types of `options` argument in `mytaskbook`.

 However, we can still pass options manually to each task as standard Python dictionaries.

In [ ]:
@task
def add(x, options: int):
    return x + options


@task
def multiply(x, options: int):
    return x * options


@taskbook
def mytaskbook(options: dict):
    add(x=1, options=options["operand1"])
    multiply(x=2, options=options["operand2"])

In [ ]:
mytaskbook(options={"operand1": 1, "operand2": 2})